# NGED LV Feeder Data Exploration

Exploring NGED's LV Feeder data to better understand it.

In our alpha prototype we noted no major issues with NGED's data, except a few
substations missing data. We want to make sure that this is still true now we're 
processing a year's worth of data, not just one month.

## Substations with missing data

It's a little tricky to explore the entire dataset immediately, because I only have 16GB 
of RAM. To arrive at the below I explored things a few months at a time and confirmed a
few things:

- No rows are missing the most important data values: substation/feeder id, consumption, 
  timestamps, number of meters
- Very few rows are missing data generally, but when they are it, it appears to be in
  the same specific columns every time: dataset_id, dno_alias, substation_geo_location

In [2]:
import pandas as pd

In [ ]:
null_substations = set()
for file in ["2024-01.parquet", "2024-02.parquet", "2024-03.parquet", "2024-04.parquet", "2024-05.parquet", "2024-06.parquet", "2024-07.parquet", "2024-08.parquet", "2024-09.parquet", "2024-10.parquet", "2024-11.parquet"]:
    df = pd.read_parquet(f"s3://weave.energy/data/staging/nged/{file}", storage_options={"profile": "weave-admin"})
    df.replace('', pd.NA, inplace=True)
    null_rows = df[df["dataset_id"].isnull()]
    print(null_rows.info())
    print(null_rows["secondary_substation_id"].unique())
    null_substations.update(set(null_rows["secondary_substation_id"].unique()))

<class 'pandas.core.frame.DataFrame'>
Index: 32722 entries, 562732 to 19932256
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   dataset_id                       0 non-null      object             
 1   dno_alias                        0 non-null      object             
 2   secondary_substation_id          32722 non-null  object             
 3   secondary_substation_name        0 non-null      object             
 4   lv_feeder_id                     32722 non-null  object             
 5   lv_feeder_name                   32722 non-null  object             
 6   substation_geo_location          0 non-null      object             
 7   aggregated_device_count_active   32722 non-null  float64            
 8   total_consumption_active_import  32722 non-null  float64            
 9   data_collection_log_timestamp    32722 non-null  datetime64[ms, UTC]


Putting together the substations which this affects in each month, we get this:

In [6]:
missing = {
    '01': ['937436', '796065', '862402', '795773', '845391', '936497', '917605', '723742', '433668', '223898', '881321', '904224', '760714', '729708', '939513'],
    '02': ['904224', '862402', '729708', '723742', '936497', '845391', '796065', '223898', '231649', '937436', '760714', '795773', '939513', '917605'],
    '03': ['760714', '862402', '729708', '223898', '453219', '917605', '936497', '904224', '903736', '764550', '742517', '796065', '918010', '943517', '939513', '723742', '936491', '881241', '756702', '845391', '795773'],
    '04': ['231649', '723742', '796065', '904224', '937436', '845391', '844072', '795773', '939513', '760714', '917605', '862402', '911843', '223898', '936497', '729708'],
    '05': ['723742', '939513', '904224', '936497', '796065', '911843', '937436', '917605', '845391', '795773', '729708', '844072', '231649', '862402', '760714'],
    '06': ['845391', '936491', '756702', '723742', '223898', '904224', '742517', '881241', '862402', '764550', '796065', '918010', '231649', '936497', '795773', '937436', '939513', '723573', '943517', '911843', '844072', '917605', '903736', '729708'],
    '07': ['862402', '795773', '742517', '903736', '723742', '723573', '939513', '937436', '911843', '729708', '881241', '845391', '764550', '942888', '918010', '844072', '796065', '936491', '936497', '453219', '943517', '904224', '756702'],
    '08': ['796065', '756702', '844744', '729708', '723742', '881241', '917605', '862402', '764550', '903736', '936491', '742517', '845391', '937436', '943517', '723573', '844072', '231649', '795773', '918010', '936497', '939513', '904224', '911843'],
    '09': ['943517', '844072', '845391', '904224', '723742', '796065', '936497', '551509', '936491', '844744', '939513', '723573', '903736', '223898', '862402', '918010', '742517', '729708', '937436', '764550', '453219', '231649', '795773', '756702', '881241'],
    '10': ['862402', '796235', '936491', '936497', '796065', '943517', '917605', '764550', '881241', '756702', '453219', '937436', '231649', '904224', '903736', '844744', '551509', '845391', '729708', '844072', '918010', '742517', '223898', '939513', '723742', '795773', '723573'],
    '11': ['844072', '936497', '939513', '742517', '904224', '223898', '917605', '453219', '729708', '795773', '862402', '845391', '881241', '764550', '943517', '756702', '796065', '723573', '903736', '231649', '796235', '937436', '936491', '844744', '723742', '918010', '551509']
}

But if we compare these lists, we find it doesn't tend to grow much - it's the same 
repeat offenders from the very first month, then (in general) one or two get added each
time.

In [7]:
prev_missing = set()
for month, subs in missing.items():
    print(f"Month {month}, new_missing: {sorted(set(subs).difference(set(prev_missing)))}")
    prev_missing.update(set(subs))


Month 01, new_missing: ['223898', '433668', '723742', '729708', '760714', '795773', '796065', '845391', '862402', '881321', '904224', '917605', '936497', '937436', '939513']
Month 02, new_missing: ['231649']
Month 03, new_missing: ['453219', '742517', '756702', '764550', '881241', '903736', '918010', '936491', '943517']
Month 04, new_missing: ['844072', '911843']
Month 05, new_missing: []
Month 06, new_missing: ['723573']
Month 07, new_missing: ['942888']
Month 08, new_missing: ['844744']
Month 09, new_missing: ['551509']
Month 10, new_missing: ['796235']
Month 11, new_missing: []


I wonder if the data we need for these "missing" substations is actually present in some 
months, but not others, and could be back-filled now we have more of it?

In [ ]:
missing_substation_ids = prev_missing.copy()
backfillable = {}
for file in ["2024-01.parquet", "2024-02.parquet", "2024-03.parquet", "2024-04.parquet", "2024-05.parquet", "2024-06.parquet", "2024-07.parquet", "2024-08.parquet", "2024-09.parquet", "2024-10.parquet", "2024-11.parquet"]:
    df = pd.read_parquet(f"s3://weave.energy/data/staging/nged/{file}", storage_options={"profile": "weave-admin"}, filters=[("secondary_substation_id", "in", missing_substation_ids), ("dataset_id", "!=", "")])
    if df.empty:
        continue
    for substation_id in missing_substation_ids.difference(set(backfillable.keys())):
        rows = df[df["secondary_substation_id"] == substation_id]
        if rows.empty:
            continue
        backfillable[substation_id] = rows.iloc[0]

In [18]:
backfillable

{'911843': dataset_id                                      NGED_911843_1_Jan_2024
 dno_alias                                                         NGED
 secondary_substation_id                                         911843
 secondary_substation_name          VILLAGE HALL WESTON HILLS - FS TRAN
 lv_feeder_id                                                         1
 lv_feeder_name                                                       1
 substation_geo_location                                52.7661, -.1081
 aggregated_device_count_active                                     8.0
 total_consumption_active_import                                  974.0
 data_collection_log_timestamp                2024-01-01 00:00:00+00:00
 insert_time                                  2024-02-06 11:02:28+00:00
 last_modified_time                           2024-03-01 13:52:37+00:00
 Name: 1488, dtype: object,
 '942888': dataset_id                            NGED_942888_1_Jan_2024
 dno_alias                

TL;DR - it is for a small handful, but not for most

## Conclusion
We could backfill three of the substations with data from other files, and we can 
recreate some of the missing columns (e.g. `dataset_id`, `dno_alias`) but on the whole 
there's still quite a lot missing. I don't have the time right now to look for other
data sources that might contain them, but we can bring this to NGED's attention when we
speak to them to see if they can suggest anything.